In [74]:
import os
import time

import tqdm
import tqdm.contrib.concurrent

import pickle
import scipy
import random
import numpy as np
from scipy.spatial import ConvexHull
from scipy.spatial.distance import pdist, squareform

import ase
import ase.io
from ase.visualize import view, plot
import matplotlib.pyplot as plt

from ppafm import io

from typing import List, Tuple

elements = ['H' , 'He',
            'Li', 'Be',  'B',  'C',  'N',  'O',  'F', 'Ne', 
            'Na', 'Mg', 'Al', 'Si',  'P',  'S', 'Cl', 'Ar',
             'K', 'Ca', 
            'Sc', 'Ti',  'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                        'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
            'Rb', 'Sr',
             'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
                        'In', 'Sn', 'Sb', 'Te',  'I', 'Xe'
]

In [59]:
# https://stackoverflow.com/a/8453514
def random_unit_vector():
    vec = np.random.normal(size=3)
    vec /= np.linalg.norm(vec)
    return vec

def plane_from_3points(points):
    a = points[1] - points[0]
    b = points[2] - points[0]
    n = np.cross(a, b)
    n /= np.linalg.norm(n)
    d = -np.dot(n, points[0])
    return np.array([n[0], n[1], n[2], d])

def plane_from_2points(points):
    p0 = points[0] + np.array([5, 5, 5])
    a = points[0] - p0
    b = points[1] - p0
    n = np.cross(a, b)
    n /= np.linalg.norm(n)
    d = -np.dot(n, points[0])
    return np.array([n[0], n[1], n[2], d])

def zyz_rotation(alpha, beta, gamma):
    '''
    Extrinsic rotation of angle alpha around z, beta around y, and gamma around z.
    '''

    def R_y(x):
        return [[np.cos(x), 0, np.sin(x)], [0, 1, 0], [-np.sin(x), 0, np.cos(x)]]
    def R_z(x):
        return [[np.cos(x), -np.sin(x), 0], [np.sin(x), np.cos(x), 0], [0, 0, 1]]

    return np.dot(R_z(gamma), np.dot(R_y(beta), R_z(alpha)))

def cart_to_sph(vec):
    r = np.linalg.norm(vec)
    phi = np.arctan2(vec[1], vec[0])
    theta = np.arccos(vec[2]/r)
    return r, phi, theta

def get_convex_hull_eqs(xyz, angle_tolerance=5):
    '''
    Get coefficients for equations of planes of convex hull of point cloud. If plane normals
    are within angle_tolerance, they are considered to be the same plane and the other plane
    is ignored.
    '''
    xyz = xyz[:,:3]
    hull = ConvexHull(xyz)
    cosines = 1 - pdist(hull.equations[:,:3], 'cosine')
    angles = np.arccos(cosines) / np.pi * 180
    angles = squareform(angles)
    bad_inds = []
    for i, angle in enumerate(angles):
        if i in bad_inds:
            continue
        inds = np.where(angle < 5)[0]
        for ind in inds:
            if ind == i:
                continue
            if ind not in bad_inds:
                bad_inds.append(ind)
    eqs = np.delete(hull.equations, bad_inds, axis=0)
    return eqs, hull

def find_planar_segments(xyz, eqs, dist_tol=0.2, num_atoms=10):
    '''
    Find planar segments on the surface of a molecule. A planar segment is
    a surface plane that contains at least num_atoms atoms within dist_tol of
    the plane.
    '''
    planar_seg_eqs = []
    planar_seg_inds = []
    for i, eq in enumerate(eqs):
        eq = eq / np.linalg.norm(eq[:3])
        dist = np.abs(np.dot(xyz[:,:3], eq[:3]) + eq[-1])
        if len(np.where(dist <= dist_tol)[0]) >= num_atoms:
            planar_seg_eqs.append(eq)
            planar_seg_inds.append(i)
    return planar_seg_eqs, planar_seg_inds

def get_plane_elements(xyz, plane_eqs, dist_tol=0.5):
    '''
    Find and count the number of atoms of different elements in a molecule near planes.
    '''
    plane_elems = []
    for eq in plane_eqs:
        eq = eq / np.linalg.norm(eq[:3])
        dist = np.abs(np.dot(xyz[:,:3], eq[:3]) + eq[-1])
        plane_elems.append(set(xyz[dist <= dist_tol, -1].astype(int)))
    return plane_elems

def _convert_elemements(element_dict):
    element_dict_ = {}
    for i, e in enumerate(element_dict):
        if isinstance(e, str):
            element_dict_[elements.index(e)+1] = element_dict[e]
        else:
            element_dict_[e] = element_dict[e]
    return element_dict_

def choose_rotations_bias(xyz, flat=True, plane_bias={'F': 1, 'Cl': 0.8, 'Br': 1},
    random_bias={'F': 0.5, 'Cl': 0.4, 'Br': 2}, angle_tolerance=5, elem_dist_tol=0.7, 
    flat_dist_tol=0.1, flat_num_atoms=10):

    n_vecs = []

    plane_bias = _convert_elemements(plane_bias)
    random_bias = _convert_elemements(random_bias)
    
    if len(xyz) > 3:
        try:
            eqs, hull = get_convex_hull_eqs(xyz, angle_tolerance=angle_tolerance)
            vertices = hull.vertices
        except scipy.spatial.qhull.QhullError:
            print(f'A problematic molecule encountered.')
            return []
    elif len(xyz) == 3:
        eqs = plane_from_3points(xyz[:,:3])[None]
        vertices = np.array([0, 1, 2])
    elif len(xyz) == 2:
        eqs = plane_from_2points(xyz[:,:3])[None]
        vertices = np.array([0, 1])
    else:
        print(xyz)
        raise RuntimeError('Molecule with less than two atoms.')
    
    if flat:
        planar_seg_eqs, planar_seg_inds = find_planar_segments(xyz, eqs, dist_tol=flat_dist_tol, num_atoms=flat_num_atoms)
        for eq in planar_seg_eqs:
            n_vecs.append(eq[:3])
        eqs = np.delete(eqs, planar_seg_inds, axis=0)

    if plane_bias:
        plane_elems = get_plane_elements(xyz, eqs, dist_tol=elem_dist_tol)
        for eq, elems in zip(eqs, plane_elems):
            for e, p in plane_bias.items():
                if e in elems and (np.random.rand() <= p):
                    n_vecs.append(eq[:3])
                    break

    if random_bias:
        elems = set(xyz[vertices,-1].astype(int))
        for e in random_bias:
            if e not in elems:
                continue
            while random_bias[e] > 0:
                if random_bias[e] < 1 and np.random.rand() > random_bias[e]:
                    break
                while True:
                    n = random_unit_vector()
                    _, phi, theta = cart_to_sph(n)
                    new_xyz = xyz.copy()
                    new_xyz[:,:3] = np.dot(new_xyz[:,:3], zyz_rotation(-phi, -theta, 0).T)
                    eq = np.array([0, 0, 1, -new_xyz[:,2].max()])
                    elems = get_plane_elements(new_xyz, [eq], dist_tol=0.7)
                    if e in elems[0]:
                        break
                if len(n_vecs) > 0:
                    n_vecs_np = np.stack(n_vecs, axis=0)
                    angles = np.arccos(np.dot(n_vecs_np, n)/np.linalg.norm(n_vecs_np, axis=1)) / np.pi * 180
                    if all(angles > angle_tolerance):
                        n_vecs.append(n)
                        random_bias[e] -= 1
                else:
                    n_vecs.append(n)
                    random_bias[e] -= 1

    rotations = []
    for vec in n_vecs:
        _, phi, theta = cart_to_sph(vec)
        rotations.append(zyz_rotation(-phi, -theta, 0))

    return rotations

In [64]:
save_dir = '/Users/kurkil1/data/rotations_04-12-24.pickle'
os.makedirs(os.path.dirname(save_dir), exist_ok=True)
database_path = '/Users/kurkil1/data/mol_database/'
filenames = [
    os.path.join(database_path, f)
    for f in os.listdir(database_path)
    if f.endswith('.xyz')
]

In [51]:
def rotate_xyz(xyz, R):
    new_xyz = xyz.copy()
    new_xyz[:,:3] = np.dot(new_xyz[:,:3], R.T)
    return new_xyz

In [84]:
def return_rotations(
    filenames: List[str],
    valid_elements: List[str],
    flat: bool = True,
    plane_bias: dict = {'F': 1, 'Cl': 0.8, 'Br': 1},
    random_bias: dict = {'F': 0.5, 'Cl': 0.4, 'Br': 2},
    angle_tolerance: float = 5,
    elem_dist_tol: float = 0.7,
    flat_dist_tol: float = 0.1,
) -> List[np.ndarray]:
    rotations = []
    for filename in tqdm.tqdm(filenames):
        xyz, zs, qs, comment = io.loadXYZ(filename)
        cid = comment.split()[-1]
        if np.any(~np.isin(zs, valid_elements)):
            continue
        rots = choose_rotations_bias(
            xyz,
            flat=flat,
            plane_bias=plane_bias,
            random_bias=random_bias,
            angle_tolerance=angle_tolerance,
            elem_dist_tol=elem_dist_tol,
            flat_dist_tol=flat_dist_tol
        )
        if len(rots) == 0:
            continue
        rotations.append({cid: rots})
    return rotations

def rotations_wrapper(*args, **kwargs):
    return return_rotations(*args, **kwargs)

In [85]:
flat = True
flat_dist_tol = 0.1
elem_dist_tol = 0.7
angle_tolerance = 5
plane_bias = {
    'H' : 0.0,
    'C' : 0.0,
    'N' : 0.1,
    'O' : 0.0,
    'F' : 1,
}
random_bias = {
    'H' : 0.1,
    'C' : 0.0,
    'N' : 0.4,
    'O' : 0.1,
    'F' : 1,
}
valid_elements = np.array([1, 6, 7, 8, 9])

# Set random seeds for reproducibility
random.seed(0)
np.random.seed(0)

In [86]:
num_workers = 8

args_list = [
    (filenames[i::num_workers], valid_elements, flat, plane_bias, random_bias, angle_tolerance, elem_dist_tol, flat_dist_tol)
    for i in range(num_workers)
]

rotations = []

tqdm.contrib.concurrent.process_map(
    rotations_wrapper, args_list, max_workers=num_workers
)

  0%|          | 0/8 [00:00<?, ?it/s]Process SpawnProcess-9:
Traceback (most recent call last):
  File "/Users/kurkil1/miniconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kurkil1/miniconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kurkil1/miniconda3/lib/python3.11/concurrent/futures/process.py", line 249, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kurkil1/miniconda3/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'rotations_wrapper' on <module '__main__' (built-in)>
Process SpawnProcess-10:
Traceback (most recent call last):
  File "/Users/kurkil1/miniconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/User

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.